# STAR-CCM+ live Job Monitoring

V1.3  

Changes from 1.2 - minimised keywords - only 1 and 3 stays to clear all lines above and below the exact log


## Install required tools

In [ ]:
%%capture
%%bash 
pip install plotly pandas ipywidgets
pip install --upgrade plotly

In [ ]:
import re
import csv
import time
import sys
import threading
import pandas as pd
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display, clear_output


## Filter data in the Log 

In [ ]:

def is_numerical(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def filter_and_save(input_file_path, output_file_path, keyword1): #keyword3
    try:
        with open(input_file_path, 'r') as input_file:
            lines = input_file.readlines()

        # Initialize flags to track whether the keywords have been found
        keyword1_found = False
        #keyword3_found = False
        filtered_lines = []

        for line in lines:
            if keyword1 in line:
                if not keyword1_found:
                    keyword1_found = True
                    filtered_lines.append(line)  # Include the first occurrence of keyword1
                else:
                    continue  # Skip subsequent occurrences of keyword1
            #elif keyword3 in line:
                #keyword3_found = True
                #break  # Stop processing lines after keyword3 is found
            
            elif keyword1_found:
                # Check if the line contains at least two columns
                columns = line.split()
                if len(columns) >= 2 and is_numerical(columns[1]):
                    filtered_lines.append(line)

        # Write the filtered lines to the new output file
        with open(output_file_path, 'w') as output_file:
            output_file.writelines(filtered_lines)

        if keyword1_found:
            print(f"Filtered content saved to '{output_file_path}' successfully.")
        else:
            print(f"Keyword '{keyword1}' not found in the file.")


        #if keyword3_found:
            #print(f"Keyword '{keyword3}' found. Content after it has been deleted.")
    
    except FileNotFoundError:
        print(f"File '{input_file_path}' not found.")

# Usage example:
input_file_path = 'master.log'
output_file_path = 'clean.log'
keyword1 = 'Iteration     Continuity     X-momentum     Y-momentum     Z-momentum'
#keyword3 = 'stopping criteria reached'
filter_and_save(input_file_path, output_file_path, keyword1) #keyword3


# Convert Log data to CSV for Plotting


In [ ]:
FIRST_CHAR_IDX = 23


def get_column_ranges(value_row):
    column_ranges = []

    in_value = False
    start_idx = 0
    current_idx = 0

    for c in value_row[FIRST_CHAR_IDX:]:
        if not in_value and c == " ":
            pass
        elif not in_value and c != " ":
            in_value = True
        elif in_value and (c == " " or c == "\n"):
            column_ranges.append((start_idx, current_idx))
            start_idx = current_idx + 1
            in_value = False

        current_idx += 1

    return column_ranges


def convert_to_csv(in_file, out_file):
    with open(in_file, "r") as f:
        lines = f.readlines()

    column_ranges = get_column_ranges(lines[1])

    with open(out_file, "w", encoding="UTF8", newline="") as f:
        writer = csv.writer(f)

        for l in lines:
            row = []
            for s, e in column_ranges:
                row.append(l[FIRST_CHAR_IDX:][s : e + 1].strip())
            writer.writerow(row)


convert_to_csv("clean.log", "clean.csv")


# Interactive Plotting


In [ ]:
# Read your data into a DataFrame
data_file = './clean.csv'
data = None  # Initialize data as None
figures = []  # Initialize a list to store figures

# Define a function to read and update data
def update_data(b=None):
    global data, figures

    # Read the new data
    new_data = pd.read_csv(data_file)

    data = new_data  # Update the global data variable

    # Update the existing figures with new data
    for fig in figures:
        update_plot(fig)

    # Refresh the dropdown options
    x_selector.options = get_column_options()
    y_selector.options = get_column_options()

# Get a list of all column names in the DataFrame
def get_column_options():
    if data is not None:
        return list(data.columns)
    return []

# Create a dropdown widget for selecting columns
x_selector = widgets.Dropdown(
    options=get_column_options(),
    description='Select X-Axis:',
)

y_selector = widgets.SelectMultiple(
    options=get_column_options(),
    description='Select Y-Axis:',
)

# Create a reload button
#reload_button = widgets.Button(
#    description='Reload Data',
#)

# Create a clear plot button
clear_plot_button = widgets.Button(
    description='Clear Plot',
)

# Create a function to clear the plot and layout
def clear_plot(b=None):
    global figures
    for fig in figures:
        fig.data = []
        # Remove the figure from the figures list
        figures.remove(fig)

# Connect the clear plot button to the click handler
clear_plot_button.on_click(clear_plot)

# Create a function to update a single plot based on the selected columns
def update_plot(fig):
    if data is None:
        return

    x_col = x_selector.value

    # Check if any Y variables are selected
    if not y_selector.value:
        with plot_output:
            plot_output.clear_output(wait=True)
            print("Please select at least one Y-axis variable.")
        return

    # Clear the existing traces
    fig.data = []

    for y_col in y_selector.value:
        fig.add_trace(go.Scatter(x=data[x_col], y=data[y_col], mode='lines+markers', name=y_col))

    # Update the layout
    fig.update_layout(
        title=f'Plot: {", ".join(y_selector.value)} vs. {x_col}',
        xaxis=dict(title=x_col),
        yaxis=dict(title="Value")
    )

# Create a plot button
plot_button = widgets.Button(
    description='Plot',
)

# Create an output widget for the plot
plot_output = widgets.Output()

# Create a function to add a new subplot
def add_subplot(b=None):
    global figures
    new_fig = go.FigureWidget()
    figures.append(new_fig)

    update_plot(new_fig)

    with plot_output:
        plot_output.clear_output()
        display(new_fig)

# Create a clear plot button
clear_plot_button = widgets.Button(
    description='Clear Plot',
)

# Connect the clear plot button to the click handler
clear_plot_button.on_click(clear_plot)

# Create a function to clear the plot
def clear_plot(b=None):
    global figures
    for fig in figures:
        fig.data = []

#def clear_plot(b=None):
#    global figures
#    for fig in figures:
#        fig.data = []

# Connect the reload button to the click handler
#reload_button.on_click(update_data)

# Connect the plot button to the click handler
plot_button.on_click(add_subplot)

# Display the widgets and the initial plot
display(x_selector, y_selector, plot_button) #reload_button
display(plot_output)  # Display the plot output widget

# Add the clear plot button to the display
display(clear_plot_button)

# Create a button to trigger the update of Sections 1 and 2
update_button = widgets.Button(
    description='Update Data and Plot',
)

# Create a function to update Sections 1 and 2 upon clicking a button
def update_sections(b=None):
    input_file_path = 'master.log'
    output_file_path = 'clean.log'
    keyword1 = 'Iteration     Continuity     X-momentum     Y-momentum     Z-momentum'

    filter_and_save(input_file_path, output_file_path, keyword1)

    in_file = 'clean.log'
    out_file = 'clean.csv'
    convert_to_csv(in_file, out_file)

    # Update the interactive plot if a layout exists
    if figures:
        update_data()

# Connect the button to the update_sections function
update_button.on_click(update_sections)

# Display the update button
display(update_button)

# Initial data load
update_data()
